In [1]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.4 MB/s eta 0:00:00


In [2]:
import os
import zipfile
import tensorflow as tf

from google.colab import drive
from keras_tuner import HyperModel, RandomSearch

from tensorflow.keras import layers, models, regularizers

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.mixed_precision import Policy, set_global_policy
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [3]:
# Información sobre el uso de la gpu
tf.debugging.set_log_device_placement(False)

In [4]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Configuración de crecimiento de memoria establecida para la GPU.")
    except RuntimeError as e:
        print(f"Error al configurar el crecimiento de memoria: {e}")

Configuración de crecimiento de memoria establecida para la GPU.


In [5]:
drive.mount('/drive')

Mounted at /drive


In [6]:
# Se extrae el archivo zip
zip_path = '/drive/MyDrive/images_processed.zip'
extract_dir = 'images_processed'

# Descomprimir el archivo ZIP
with zipfile.ZipFile(zip_path, 'r') as zip_file:
    zip_file.extractall(extract_dir)

In [7]:
# Tamaño de la imagen y las categorías
img_height = 64
img_width = 64
batch_size = 64
num_categories = 10

In [8]:
def preprocess_input(image):
    # Centra y escala la imagen
    return (image - 127.5) / 255.0

In [9]:
# Directorio
extract_dir = 'images_processed'
# Lista todas las clases disponibles
all_classes = sorted(os.listdir(extract_dir))

# Selecciona las primeras clases
selected_classes = all_classes[:num_categories]

In [10]:
print(f"Clases seleccionadas: {selected_classes}")

Clases seleccionadas: ['Amethyst_0', 'Azurite_1', 'Calcite_2', 'Copper_3', 'Fluorite_4', 'Malachite_5', 'Pyrite_6', 'Pyromorphite_7', 'Quartz_8', 'Wulfenite_9']


In [11]:
# Usar un generador personalizado
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.1,
    rotation_range=10,           # Rotaciones aleatorias
    width_shift_range=0.1,       # Desplazamiento horizontal aleatorio
    height_shift_range=0.1,      # Desplazamiento vertical aleatorio
    shear_range=0.1,             # Transformaciones de corte
    zoom_range=0.1,              # Zoom aleatorio
    horizontal_flip=True,        # Volteo horizontal
    fill_mode='nearest'          # Relleno de píxeles
)

# Generador de imágenes para el conjunto de entrenamiento
train_generator = train_datagen.flow_from_directory(
    extract_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    classes=selected_classes,
    shuffle=True
)

# Generador de imágenes para el conjunto de validación
validation_generator = train_datagen.flow_from_directory(
    extract_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    classes=selected_classes,
    shuffle=False
)

Found 45000 images belonging to 10 classes.
Found 5000 images belonging to 10 classes.


In [12]:
# Imprime la información sobre las clases
print("Class mapping:", train_generator.class_indices)

Class mapping: {'Amethyst_0': 0, 'Azurite_1': 1, 'Calcite_2': 2, 'Copper_3': 3, 'Fluorite_4': 4, 'Malachite_5': 5, 'Pyrite_6': 6, 'Pyromorphite_7': 7, 'Quartz_8': 8, 'Wulfenite_9': 9}


In [13]:
# Convierte el generador en un tf.data.Dataset para agregar repetición y precarga
train_dataset = tf.data.Dataset.from_generator(
    lambda: train_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, img_height, img_width, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, num_categories), dtype=tf.float32)
    )
)

validation_dataset = tf.data.Dataset.from_generator(
    lambda: validation_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, img_height, img_width, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, num_categories), dtype=tf.float32)
    )
)

train_dataset = train_dataset.repeat()
validation_dataset = validation_dataset.repeat()

train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
validation_dataset = validation_dataset.prefetch(tf.data.AUTOTUNE)

In [14]:
# Usa precisión mixta
policy = Policy('mixed_float16')
set_global_policy(policy)
print(f"Política de precisión actual: {tf.keras.mixed_precision.global_policy()}")

Política de precisión actual: <DTypePolicy "mixed_float16">


In [15]:
# Definición del modelo con Keras Tuner para optimización de hiperparámetros
class MyHyperModel(HyperModel):
    def build(self, hp):
        model = models.Sequential([

            # Capa 1
            layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(img_height, img_width, 3)),
            layers.MaxPooling2D((2, 2)),

            # Capa 2
            layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
            layers.MaxPooling2D((2, 2)),

            # Capa 3
            layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
            layers.MaxPooling2D((2, 2)),

            # Capa 4
            layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
            layers.MaxPooling2D((2, 2)),

            # Capa 5
            layers.Conv2D(1024, (3, 3), activation='relu', padding='same'),
            layers.MaxPooling2D((2, 2)),

            # Aplanamiento
            layers.GlobalAveragePooling2D(),

            # Capa densa con dropout y regularización L2
            layers.Dense(
            hp.Choice('units', values=[512, 1024]),
            activation='relu',
            kernel_regularizer=regularizers.l2(hp.Float('l2', min_value=1e-5, max_value=1e-1, sampling='log'))
            ),
            layers.Dropout(rate=hp.Float('dropout', min_value=0.1, max_value=0.5, step=0.1)),

            # Capa de salida
            layers.Dense(num_categories, activation='softmax')
        ])

        # Compilación del modelo
        model.compile(
            optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        return model

In [16]:
# Configuración de Keras Tuner
hypermodel = MyHyperModel()

In [17]:
# Realiza la búsqueda aleatoria de hiperparámetros
tuner = RandomSearch(
    hypermodel,
    objective='val_accuracy',
    max_trials=8,
    executions_per_trial=1,
    directory='keras_tuning',
    project_name='hyperparameter_tuning'
)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [18]:
# Realiza la búsqueda de hiperparámetros
tuner.search(
    train_dataset,
    epochs=10,
    validation_data=validation_dataset,
    validation_steps=validation_generator.samples // batch_size,
    steps_per_epoch=train_generator.samples // batch_size,
)

Trial 8 Complete [00h 18m 09s]
val_accuracy: 0.6134521961212158

Best val_accuracy So Far: 0.6213533282279968
Total elapsed time: 02h 25m 27s


In [19]:
# Muestra los resultados de todas las combinaciones de hiperparámetros
for trial in tuner.oracle.trials.values():
    trial_hps = trial.hyperparameters.values
    val_accuracy = trial.score
    print(f"Hiperparámetros: {trial_hps}")
    print(f"Precisión en validación: {val_accuracy}")
    print("---------------")

Hiperparámetros: {'units': 512, 'l2': 0.03412201943829172, 'dropout': 0.5}
Precisión en validación: 0.6112236380577087
---------------
Hiperparámetros: {'units': 512, 'l2': 3.173510685995567e-05, 'dropout': 0.30000000000000004}
Precisión en validación: 0.6193273663520813
---------------
Hiperparámetros: {'units': 1024, 'l2': 0.02141470614184636, 'dropout': 0.1}
Precisión en validación: 0.6191247701644897
---------------
Hiperparámetros: {'units': 512, 'l2': 0.0049156346408599895, 'dropout': 0.5}
Precisión en validación: 0.6173014640808105
---------------
Hiperparámetros: {'units': 1024, 'l2': 0.0006477101798213585, 'dropout': 0.1}
Precisión en validación: 0.6213533282279968
---------------
Hiperparámetros: {'units': 1024, 'l2': 4.7266372509037394e-05, 'dropout': 0.30000000000000004}
Precisión en validación: 0.6154780983924866
---------------
Hiperparámetros: {'units': 512, 'l2': 0.020875682692884864, 'dropout': 0.30000000000000004}
Precisión en validación: 0.6185170412063599
----------